### Specify the path to the dataset you want to modfiy and where to save it


In [7]:
# don't forget to run the helper function cells first
# dataset_path = "/data/mm12191/datasets/dataset_batch550000-838143.pkl"
dataset_path = "/data/kb4083/datasets/sample_20m_test_even_smaller.pkl"
# path where the modified dataset should be saved and
#the name of file without the extension
save_path = "/data/kb4083/datasets/sample_20m_test_even_smaller_modified"
new_extension = "pkl"

### Read the current dataset

In [2]:
import json
import pickle
# in case the dataset was stored in the json format
if dataset_path.endswith("json"):
    # we open the dataset as a normal file
    with open(dataset_path, "r") as f:
        dataset_str = f.read()
    programs_dict = json.loads(dataset_str)
# in case the dataset was stored in the pkl format
elif dataset_path.endswith("pkl"):
    # we un-pickle the file using the pickle library 
    with open(dataset_path, "rb") as f:
        programs_dict = pickle.load(f)

### Apply modifications

In [6]:
# shuffle the programs of the dataset
# Maximum number of nested loops in a program
max_depth = 5
# Number of all the dropped schedules
nb_dropped = 0
nb_dropped_random_matrix = 0
# Number of dropped schedules due to the drop schedule function only
nb_pruned = 0
# List of dropped functions 
dropped_funcs = []

functions_list = list(programs_dict.keys())
new_programs_dict = {}
for index, function_name in enumerate(tqdm(functions_list)):
    # Check whether this function should be dropped
    if drop_program(programs_dict[function_name], function_name):
        nb_dropped += len(
            programs_dict[function_name]["schedules_list"]
        )
        dropped_funcs.append(function_name)
        continue


    program_json = programs_dict[function_name]["program_annotation"]
    
    # Adding expression representation
    program_json = add_expression_representation_to_function(program_json, function_name)

    # Check whether the whole function should be dropped because of the number of loops and accesses
    try:
        check_program_access_and_depth(
            programs_dict[function_name],
            max_depth=max_depth,
        )
    except (NbAccessException, LoopsDepthException):
        # If one of the two exceptions was raised, we drop all the schedules for that program and skip to the next program.
        nb_dropped += len(
            programs_dict[function_name]["schedules_list"]
        )
        continue
    # Get the initial execution time for the program to calculate the speedups (initial exec time / transformed exec time)
    program_exec_time = programs_dict[function_name][
        "initial_execution_time"
    ]

    new_programs_dict[function_name] = programs_dict[function_name].copy()
    new_programs_dict[function_name]["schedules_list"] = []
    
    for iterator in new_programs_dict[function_name]["program_annotation"]["iterators"]:
        new_programs_dict[function_name]["program_annotation"]["iterators"][iterator]["upper_bound"] = str(new_programs_dict[function_name]["program_annotation"]["iterators"][iterator]["upper_bound"])
        new_programs_dict[function_name]["program_annotation"]["iterators"][iterator]["lower_bound"] = str(new_programs_dict[function_name]["program_annotation"]["iterators"][iterator]["lower_bound"])
        
    # For each schedule (sequence of transformations) collected for this function
    for schedule_index in range( len(programs_dict[function_name]["schedules_list"])):

        # Get the schedule JSON representation
        schedule_json = programs_dict[function_name]["schedules_list"][schedule_index]

        # Get the transformed execution timeschedule_index
        sched_exec_time = np.min(schedule_json["execution_times"])

        # Check if this schedule should be dropped
        if drop_schedule(programs_dict[function_name], schedule_index ) or (not sched_exec_time):
            nb_dropped += 1
            nb_pruned += 1
            continue
        # Calculate the speed up obtained from applying the list of transformations spesified by the schedule
        sched_speedup = program_exec_time / sched_exec_time

        # Check whether we can set a default value for this speedup through the can_set_default_eval function.
        def_sp = can_set_default_eval(program_json, schedule_json)


        # If the function returns 0, this means no default value was spesified
        if def_sp > 0:
            sched_speedup = def_sp
            

        # Check whether we can clip the obtained speedup
        sched_speedup = speedup_clip(sched_speedup)
        
        # TODO add speedupclip

        # Fill the obtained template with the corresponsing schedule features
        try:
            comp_transformations = get_transformations_list(
                program_json,
                schedule_json,
                max_depth,
            )
        except NbTranformationException:
            # If the number of transformations exceeded the specified max, we skip this schedule
            nb_dropped += 1
            continue

        except RandomMatrix :
            nb_dropped += 1
            nb_dropped_random_matrix += 1
            continue
        computations_dict = program_json["computations"]
        ordered_comp_list = sorted(
            list(computations_dict.keys()),
            key=lambda x: computations_dict[x]["absolute_order"],
        )
        
        schedule_json["legality_check"] = True
        schedule_json["exploration_method"] = 1
        for comp_index, comp_name in enumerate(ordered_comp_list):
            if("interchange_dims" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("interchange_dims", None)
            
            if("unfuse_iterators" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("unfuse_iterators", None)
                
            if("average_skewed_extents" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("average_skewed_extents", None)
                
            if("skewing" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("skewing", None)

            if("Transformation Matrix" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("Transformation Matrix", None)

            if("transformation_matrix" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("transformation_matrix", None)

            if("transformation_matrices" in schedule_json[comp_name]):
                remove_key = schedule_json[comp_name].pop("transformation_matrices", None)
                
            if("shiftings" not in schedule_json[comp_name]):
                schedule_json[comp_name]["shiftings"] = None
                
            if(abs(comp_transformations[comp_index]).sum() != 0):
                schedule_json[comp_name]["transformations_list"] = comp_transformations[comp_index].tolist()[::-1]
            else:
                schedule_json[comp_name]["transformations_list"] = []
            
        schedule_json["sched_str"] = get_schedule_str(program_json, schedule_json)
        new_programs_dict[function_name]["schedules_list"].append(schedule_json)
        
    # Adding roots table to the program tree structure
    if "roots" not in programs_dict[function_name]["schedules_list"][0]["tree_structure"]:
        for i in range(len(programs_dict[function_name]["schedules_list"])):
            programs_dict[function_name]["schedules_list"][i]["tree_structure"] = {"roots": [programs_dict[function_name]["schedules_list"][i]["tree_structure"]]}


100%|██████████| 91/91 [00:05<00:00, 16.68it/s]


### Save the modified dataset

In [8]:
path = save_path +"."+new_extension
if(new_extension == "json"):
    with open(path, "w") as outfile:
        json.dump(new_programs_dict, outfile)
if(new_extension == "pkl"):
    with open(path, 'wb') as handle:
        pickle.dump(new_programs_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Helper functions

In [3]:
import copy
import enum
import random
import re
import numpy as np
import pandas as pd
import json
import torch
from tqdm import tqdm
# An exception to limit the maximum number of allowed transformations 
class NbTranformationException(Exception):
    pass

class RandomMatrix(Exception):
    pass

# An exception to limit the maximum number of read-write accesses. 
class NbAccessException(Exception):
    pass

# An exception to limit the maximum number of nested loops. Currently set to 5.
class LoopsDepthException(Exception):
    pass

# An exception that's raised when we don't find the expression tree representation
class ComputationExpressionException(Exception):
    pass

# Maximum sequence of transformations (reversal, interchange and skewing) allowed. Currently set to 5 
MAX_NUM_TRANSFORMATIONS = 4

# Maximum size of the tags vector representing each transformation
MAX_TAGS = 8

# Enumeration for the different exploration algorithms used to generate the data
# class Exploration_method(int, enum.Enum):
#    Beam_search = 0
#    Recursive_beam_search = 1
#    Reinforcement_learning = 2

def get_transformations_list(
    program_json, schedule_json, max_depth,
):
    
    computations_dict = program_json["computations"]
    ordered_comp_list = sorted(
        list(computations_dict.keys()),
        key=lambda x: computations_dict[x]["absolute_order"],
    )
    
    transformations_list = []
    for comp_index, comp_name in enumerate(ordered_comp_list):
        comp_dict = program_json["computations"][comp_name]
        comp_schedule_dict = schedule_json[comp_name]
        
        # Check which transformations (interchange, reversal and skweing) were applied and add the padded vector representation to their corresponding position
        padded_tags = get_transformation_tags(
            program_json, schedule_json, comp_name, max_depth
        )
        transformations_list.append(padded_tags)

    return transformations_list

# A function to extract the transformations applied on a spesific computation in the form of a vector of tags
# Padding is added if the number of transformations is less than the maximum value of MAX_NUM_TRANSFORMATIONS
# Currently our dataset represents transformations in two different formats.
#         1- in the form of matrices from the polyhedral representation
#         2- in the form of tags for each transformation
# We generated a variaty of representations to test which one is more useful for our spesfici usage
# In this function we will be unifying all of the dataset into the tags representation 
# The tag representation is as follows:
#         ['type_of_transformation', 'first_interchange_loop', 'second_interchange_loop', 'reversed_loop', 'first_skewing_loop', 'second_skewing_loop', 'first_skew_factor', 'second_skew_factor']
#     Where the type_of_transformation tag is:
#         - 0 for no transformation being applied
#         - 1 for loop interchange
#         - 2 for loop reversal
#         - 3 for loop skewing

def get_transformation_tags(
    program_json, schedule_json, comp_name, max_depth=None
):
    # Extract information about the computation and the transformations that were applied from the json input
    comp_dict = program_json["computations"][comp_name]
    comp_schedule_dict = schedule_json[comp_name]
    nb_iterators = len(comp_dict["iterators"])
    loop_nest = comp_dict["iterators"][:]
    
    # Create an identity vector that represents that no transformation was applied
    identity = np.zeros((nb_iterators, nb_iterators), int)
    np.fill_diagonal(identity, 1)
    identity_tags = np.zeros((1,MAX_TAGS), dtype=np.int32)
    
    tag_factors = []
    
    # If the transformations are represented using matrices
    if "transformation_matrices" in comp_schedule_dict:
        
        if comp_schedule_dict["transformation_matrices"] != []:
            if ("transformation_matrix" in comp_schedule_dict) and (
                comp_schedule_dict["transformation_matrix"]
            ):
                # transformation_matrix represents all of the applied transformations in a single compact matrix
                final_transformation_matrix = np.array(
                    list(map(int, comp_schedule_dict["transformation_matrix"]))
                ).reshape(nb_iterators, nb_iterators)
            else:
                final_transformation_matrix = identity.copy()

            
            tag_factors = []
            # For each transformation in the schedule
            for matrix in comp_schedule_dict["transformation_matrices"][::-1]:
                # Check whether the size of the matrix is coherent
                assert np.sqrt(len(matrix)) == nb_iterators
                transformation_matrix = np.array(list(map(int, matrix))).reshape(
                    nb_iterators, nb_iterators
                )
                tags_vector = identity_tags.copy()
                # Calculate the residual to check whether this transformation is the identity matrix
                residual = np.abs(identity - transformation_matrix)
                
                mask_line = residual.sum(axis=1) > 0
                mask_col  = residual.sum(axis=0) > 0
                
                non_zeros_positions = np.argwhere(residual>0)

                for index in non_zeros_positions:
                    if (abs(index[0] - index[1]) > 1) and nb_iterators != abs(transformation_matrix).sum():

                        raise RandomMatrix
                    else:
                        if (index[1] < index[0]):
                            
                            first_factor = transformation_matrix[index[0]-1][index[1]]
                            second_factor = transformation_matrix[index[0]-1][index[1]+1]

                            if ( index[1] < index[0] and first_factor == 1 and second_factor == 0 ):

                                raise RandomMatrix
                
                # If that's the case, skip it to avoid adding it in the representation
                # if (transformation_matrix == identity).all():
                if (transformation_matrix == identity).all():
                    continue
                    
                # If the transformation isn't the identity, fill the tag_vector with the corresponsonding transformation info
                elif residual.sum() != 0 and nb_iterators == transformation_matrix.sum():
                    # If the sum of the elements in the matrix is equal to the number of iterators, then interchange has been applied
                    tags_vector[0][0] = 1
                    
                    # Extract the interchange parameters (loop indecies) through the created mask
                    dims = np.arange((nb_iterators),dtype=int)[mask_line]
                    
                    assert dims.shape[0] == 2
                    first_iter_index, second_iter_index=dims
                    
                    # Add the interchange parameters to the tag vector
                    tags_vector[0][1], tags_vector[0][2] = first_iter_index, second_iter_index
                
                elif nb_iterators - 2 == np.trace(transformation_matrix) and transformation_matrix.sum() - np.trace(transformation_matrix) == 0:  
                    # If the sum of the elements in the matrix is equal to the number of iterators -2, then loop reversal has been applied
                    tags_vector[0][0] = 2      
                    
                    # Extract the reversed loop index
                    dims = np.arange((nb_iterators),dtype=int)[mask_line]
#                     print(transformation_matrix)
                    assert dims.shape[0] == 1
                    index=dims[0]
                    
                    # Add the reversal parameter to the tags vector
                    tags_vector[0][3] = index
                else:
                    # If the matrix isn't one of the first three cases (identity, reversal, interchange) then we know it represents skewing
                    tags_vector[0][0] = 3
                    dims_line = np.arange((nb_iterators),dtype=int)[mask_line]
                    dims_col = np.arange((nb_iterators),dtype=int)[mask_col]
                    
                    dims_line = [dims_line[0], dims_line[0]+1]
                    
                    dims_col = [dims_col[0], dims_col[0]+1]
                    
                    # Extract which loops have been skewed 
                    first_iter_index, second_iter_index = dims_line
                    
                    # Add the skewed loops indecies to the tags vector
                    tags_vector[0][4], tags_vector[0][5] = first_iter_index, second_iter_index

                    
                    # Extract the skewing factors 
                    first_factor = transformation_matrix[first_iter_index, first_iter_index]
                    second_factor = transformation_matrix[first_iter_index, second_iter_index]
                    a = transformation_matrix[second_iter_index, first_iter_index]
                    b = transformation_matrix[second_iter_index, second_iter_index]
                    
        
                    if ((a, b) != linear_diophantine_default(first_factor, second_factor)):

                        raise RandomMatrix
                    
                    # Add the skewing factors to the tags vector
                    tags_vector[0][6], tags_vector[0][7] = first_factor, second_factor
                
                transformation_matrix = np.c_[
                    np.ones(transformation_matrix.shape[0]), transformation_matrix
                ]
                transformation_matrix = np.r_[
                    [np.ones(transformation_matrix.shape[1])], transformation_matrix
                ]
                transformation_matrix = np.pad(
                    transformation_matrix,
                    [
                        (0, max_depth + 1 - transformation_matrix.shape[0]),
                        (0, max_depth + 1 - transformation_matrix.shape[1]),
                    ],
                    mode="constant",
                    constant_values=0,
                )
                tag_factors.append(tags_vector)
            
            # We use MAX_NUM_TRANSFORMATIONS+1 instead of MAX_NUM_TRANSFORMATIONS to include the matrix that represents the whole sequence
            if len(tag_factors) > (MAX_NUM_TRANSFORMATIONS):
                # If the number of transformations is greater than the maximum allowed, raise an exception
                raise NbTranformationException
    
    
            # If no tranformation was found (comp_schedule_dict["transformation_matrices"] only contains the identity) then we use the idendity tags
            final_tags = (
                np.concatenate(tag_factors, axis=0)
                if tag_factors
                else identity_tags.copy()
            )
        else:
            # If comp_schedule_dict["transformation_matrices"] is empty we also use the identity tags 
            final_transformation_matrix = identity.copy()
            final_tags = identity_tags.copy()
        # To make sure the data is coherent, we want to check whether the sequence of transformations is equal to the final transformation matrix (represented by final_transformation_matrix) 
        # In the polyhedral model, the multiplication of the sequence of transformation matrices results in one matrix that represents the whole combination
        comparison_matrix = identity.copy()
        for mat in comp_schedule_dict["transformation_matrices"][::-1]:
            comparison_matrix = comparison_matrix @ np.array(
                list(map(int, mat))
            ).reshape(nb_iterators, nb_iterators)
        
        assert (comparison_matrix == final_transformation_matrix).all()
    else:
        # If the non-polyhedral representation is used, only interchange and skewing are applied
        # The skewing and interchange parameters are present in the json
        # We directly add them to the tags vector 
        if comp_schedule_dict["interchange_dims"]:
            first_iter_index = loop_nest.index(
                comp_schedule_dict["interchange_dims"][0]
            )
            second_iter_index = loop_nest.index(
                comp_schedule_dict["interchange_dims"][1]
            )
            loop_nest[first_iter_index], loop_nest[second_iter_index] = (
                loop_nest[second_iter_index],
                loop_nest[first_iter_index],
            )
            tags_vector = identity_tags.copy()
            tags_vector[0][0] = 1
            tags_vector[0][1], tags_vector[0][2] = first_iter_index, second_iter_index
            tag_factors.append(tags_vector)

        
        if comp_schedule_dict["skewing"]:
            first_iter_index = loop_nest.index(
                comp_schedule_dict["skewing"]["skewed_dims"][0]
            )
            second_iter_index = loop_nest.index(
                comp_schedule_dict["skewing"]["skewed_dims"][1]
            )
            first_factor = int(comp_schedule_dict["skewing"]["skewing_factors"][0])
            second_factor = int(comp_schedule_dict["skewing"]["skewing_factors"][1])

            tags_vector = identity_tags.copy()
            tags_vector[0][0] = 3
            tags_vector[0][4], tags_vector[0][5] = first_iter_index, second_iter_index
            tags_vector[0][6], tags_vector[0][7] = first_factor, second_factor
            tag_factors.append(tags_vector)
        # If neither skewing or interchange were applied, we use the identity tags vector
        final_tags = (
            np.concatenate(tag_factors, axis=0)
            if tag_factors
            else identity_tags.copy()
        )
        
    return final_tags
# returns a string representation of a schedule and the transformations applied in it
def get_schedule_str(program_json, sched_json):
    comp_name = [
        n
        for n in sched_json.keys()
        if not n in ["unfuse_iterators", "tree_structure", "execution_times", "fusions", "sched_str", "legality_check", "exploration_method"]
    ]
    sched_str = ""
    
    if ("fusions" in sched_json and sched_json["fusions"]):
        for fusion in sched_json["fusions"]:
            sched_str += "F("
            for name in comp_name:
                if name in fusion:
                    sched_str += name + ","
            
            sched_str = sched_str[:-1]
            sched_str += ")"
            
    for name in comp_name:
        transf_loop_nest = program_json["computations"][name]["iterators"].copy()
        schedule = sched_json[name]
        sched_str += '{' + name + '}:'

        for transformation in schedule["transformations_list"]:

            if (transformation[0] == 1):
                sched_str += "I(L" + str(transformation[1]) + ",L" + str(transformation[2]) + ")"
                
            elif (transformation[0] == 2):
                sched_str += "R(L" + str(transformation[3])+ ")"
            elif (transformation[0] == 3):
                sched_str += "S(L" + str(transformation[4]) + ",L" + str(transformation[5]) + "," + str(transformation[6]) + "," + str(transformation[7]) + ")"
                
        if schedule["parallelized_dim"]:
            
            dim_index = transf_loop_nest.index(schedule["parallelized_dim"])
            sched_str += "P(L" + str(dim_index) + ")"

        if schedule["tiling"]:
            if schedule["tiling"]["tiling_depth"] == 2:
                first_dim = schedule["tiling"]["tiling_dims"][0]
                second_dim = schedule["tiling"]["tiling_dims"][1]
                
                first_dim_index = transf_loop_nest.index(first_dim)
                second_dim_index = transf_loop_nest.index(second_dim)
                first_factor = schedule["tiling"]["tiling_factors"][0]
                second_factor = schedule["tiling"]["tiling_factors"][1]
                sched_str += (
                    "T2(L"
                    + str(first_dim_index)
                    + ",L"
                    + str(second_dim_index)
                    + ","
                    + str(first_factor)
                    + ","
                    + str(second_factor)
                    + ")"
                )
                i = transf_loop_nest.index(first_dim)
                transf_loop_nest[i : i + 1] = first_dim + "_outer", second_dim + "_outer"
                i = transf_loop_nest.index(second_dim)
                transf_loop_nest[i : i + 1] = first_dim + "_inner", second_dim + "_inner"
            else:
                first_dim = schedule["tiling"]["tiling_dims"][0]
                second_dim = schedule["tiling"]["tiling_dims"][1]
                third_dim = schedule["tiling"]["tiling_dims"][2]
                first_dim_index = transf_loop_nest.index(first_dim)
                second_dim_index = transf_loop_nest.index(second_dim)
                third_dim_index = transf_loop_nest.index(third_dim)
                first_factor = schedule["tiling"]["tiling_factors"][0]
                second_factor = schedule["tiling"]["tiling_factors"][1]
                third_factor = schedule["tiling"]["tiling_factors"][2]
                sched_str += (
                    "T3(L"
                    + str(first_dim_index)
                    + ",L"
                    + str(second_dim_index)
                    + ",L"
                    + str(third_dim_index)
                    + ","
                    + str(first_factor)
                    + ","
                    + str(second_factor)
                    + ","
                    + str(third_factor)
                    + ")"
                )
                i = transf_loop_nest.index(first_dim)
                transf_loop_nest[i : i + 1] = (
                    first_dim + "_outer",
                    second_dim + "_outer",
                    third_dim + "_outer",
                )
                i = transf_loop_nest.index(second_dim)
                transf_loop_nest[i : i + 1] = (
                    first_dim + "_inner",
                    second_dim + "_inner",
                    third_dim + "_inner",
                )
                transf_loop_nest.remove(third_dim)

        if schedule["unrolling_factor"]:
            dim_index = len(transf_loop_nest) - 1
            dim_name = transf_loop_nest[-1]
            sched_str += "U(L" + str(dim_index) + "," + schedule["unrolling_factor"] + ")"
            transf_loop_nest[dim_index : dim_index + 1] = (
                dim_name + "_Uouter",
                dim_name + "_Uinner",
            )
    return sched_str

# Creates a template for the input representation
def check_program_access_and_depth(program_dict, max_depth):
    # Set the max and min number of accesses allowed 
    max_accesses = 15
    min_accesses = 1

    # Get the program JSON represenation
    program_json = program_dict["program_annotation"]
    
    # Get the computations (program statements) dictionary and order them according to the absolute_order attribute
    computations_dict = program_json["computations"]
    ordered_comp_list = sorted(
        list(computations_dict.keys()),
        key=lambda x: computations_dict[x]["absolute_order"],
    )

    for comp_index, comp_name in enumerate(ordered_comp_list):
        
        comp_dict = computations_dict[comp_name]
        
        if len(comp_dict["accesses"]) > max_accesses:
            raise NbAccessException
        
        if len(comp_dict["accesses"]) < min_accesses:
            raise NbAccessException
        
        if len(comp_dict["iterators"]) > max_depth:
            raise LoopsDepthException
    
    return

def has_skippable_loop_1comp(
    prog_dict,
):

    program_json = prog_dict["program_annotation"]
    if not len(program_json["computations"]) == 1:
        return False
    comp_name = list(program_json["computations"].keys())[0]
    comp_dict = program_json["computations"][comp_name]
    write_buffer_id = comp_dict["write_buffer_id"]
    iterators = comp_dict["iterators"]
    write_dims = isl_to_write_dims(comp_dict["write_access_relation"])
    read_buffer_ids = [e["buffer_id"] for e in comp_dict["accesses"]]

    if len(write_dims) == len(iterators):

        if (
            len(read_buffer_ids) == 1
            and read_buffer_ids[0] == write_buffer_id
            and comp_dict["number_of_additions"] == 0
            and comp_dict["number_of_subtraction"] == 0
            and comp_dict["number_of_multiplication"] == 0
            and comp_dict["number_of_division"] == 0
        ):
            return True
        return False

    if not write_buffer_id in read_buffer_ids:
        return True

    found = False
    for access in comp_dict["accesses"]:
        if access["buffer_id"] == write_buffer_id and not access_is_stencil(access):
            found = True
            break
    if not found:
        if write_dims[-1] != iterators[-1]:
            return True

    for access in comp_dict["accesses"]:
        if access["buffer_id"] == write_buffer_id and access_is_stencil(access):
            return False

    read_dims_bools = []
    for access in comp_dict["accesses"]:
        read_dims_bools.append(np.any(access["access_matrix"], axis=0))
    read_dims_bools = np.any(read_dims_bools, axis=0)
    read_iterators = [
        iterators[i]
        for i, is_used in enumerate(read_dims_bools[:-1])
        if is_used == True
    ]
    used_iterators = set(write_dims + read_iterators)
    if len(used_iterators) == len(iterators):
        return False

    if iterators[-1] in used_iterators:
        if len(comp_dict["accesses"]) > 2:
            return False

    return True

def drop_program(prog_dict, prog_name):
#     print(prog_name[8:])
    if len(prog_dict["schedules_list"]) < 2:
        return True
    if ( 750000 <= int(prog_name[8:]) and int(prog_name[8:])<=752600 ):
    
#         print("Found an random matrix program", prog_name)
        return True
    if has_skippable_loop_1comp(prog_dict):
        return True
    if (
        "node_name" in prog_dict and prog_dict["node_name"] == "lanka24"
    ):  # drop if we the program is run by lanka24 (because its measurements are inacurate)
        return True
    return False

def drop_schedule(prog_dict, schedule_index):
    schedule_json = prog_dict["schedules_list"][schedule_index]
    if (not schedule_json["execution_times"]) or min(
        schedule_json["execution_times"]
    ) < 0:  # exec time is set to -1 on datapoints that are deemed noisy, or if list empty
        return True
    if sched_is_prunable(prog_dict["program_annotation"], schedule_json):
            return True
    if wrongly_set_to_default_schedule(prog_dict, schedule_index):
        return True

    return False

def speedup_clip(speedup):
    if speedup < 0.01:
        speedup = 0.01
    return speedup

# TODO what does this function do
def isl_to_write_dims(
    isl_map,
):
    buffer_iterators_str = re.findall(r"->\s*\w*\[(.*)\]", isl_map)[0]
    buffer_iterators_str = re.sub(r"\w+'\s=", "", buffer_iterators_str)
    buf_iter_names = re.findall(r"(?:\s*(\w+))+", buffer_iterators_str)
    return buf_iter_names

# returns a string representation of a schedule and the transformations applied in it

def wrongly_set_to_default_schedule(program_dict, schedule_index):
    
    schedule_dict = program_dict["schedules_list"][schedule_index]
    if len(schedule_dict["execution_times"]) == 1:
        speed_up = program_dict["initial_execution_time"] / schedule_dict["execution_times"][0]
        
        if (speed_up > 0.00099 and speed_up < 0.00101) and (not can_set_default_eval(program_dict["program_annotation"], schedule_dict)):
                return True
    return False

def access_is_stencil(access):
    return np.any(access["access_matrix"], axis=0)[-1]

# Solving the Linear Diophantine equation & finding basic solution (sigma & gamma) for : f_i* sigma - f_j*gamma = 1
# Used to get skewing parameters
def linear_diophantine_default(f_i, f_j):
    n1 = abs(f_i)
    n2 = abs(f_j)
    
    while(n1 != n2):
        if(n1 > n2):
            n1 -=  n2
        else:
            n2 -=  n1
            
    # Update f_i and f_j to equivalent but prime between themselfs value
    f_i = f_i / n1
    f_j = f_j / n1
    
    found = False
    gamma = 0
    sigma = 1
    
    if (f_j == 1) or (f_i == 1):
        gamma = f_i - 1
        sigma = 1
        # Since sigma = 1  then
        # f_i - gamma * f_j = 1 & using the previous condition :
        #  - f_i = 1 : then gamma = 0 (f_i-1) is enough
        #  - f_j = 1 : then gamma = f_i -1  
    else:
        if (f_j == -1) and (f_i > 1):
            gamma = 1
            sigma = 0
        else:
            # General case : solving the Linear Diophantine equation & finding basic solution (sigma & gamma) for : f_i* sigma - f_j*gamma = 1
            i = 0
            while (i < 100) and (not found):
                if ((sigma * f_i) % abs(f_j)) == 1:
                    found = True
                else:
                    sigma += 1
                    i += 1
            if not found:
                # Detect infinite loop and prevent it in case where f_i and f_j are not prime between themselfs
                print("Error cannof find solution to diophantine equation")
                return
            gamma = ((sigma * f_i) - 1) / f_j
    return gamma, sigma

def sched_is_prunable(program_json, schedule_json):
    reg = ""
    comp_names = [
        n
        for n in schedule_json.keys()
        if not n in ["unfuse_iterators", "tree_structure", "execution_times", "fusions", "sched_str", "legality_check", "exploration_method"]
    ]
    for name in comp_names:
        innermost_loop = len(program_json["computations"][name]["iterators"])-1
        reg += f"{{{name}}}:P\(L{innermost_loop}\)U.*|"
    if(len(comp_names)>0):
        reg = reg[:-1]
    
    schedule_str = get_schedule_str_for_pruning(program_json, schedule_json)
    if re.search(reg, schedule_str):
        return True
    reg = ""
    
    for name in comp_names:
        innermost_loop = len(program_json["computations"][name]["iterators"])-1
        reg += f"{{{name}}}:P\(L{innermost_loop}\)T2\(L{innermost_loop-2},L{innermost_loop-1}.*|"
    
    if(len(comp_names)>0):
        reg = reg[:-1]
    if re.search(reg, schedule_str):
        return True                                                                                                                               
    return False

def can_set_default_eval(program_json, schedule_json):
    reg = ""
    comp_names = [
        n
        for n in schedule_json.keys()
        if not n in ["unfuse_iterators", "tree_structure", "execution_times", "fusions", "sched_str", "legality_check", "exploration_method"]
    ]
    for name in comp_names:
        innermost_loop = len(program_json["computations"][name]["iterators"])-1
        reg += f"{{{name}}}:P\(L{innermost_loop}\)({{|$)|"
    if(len(comp_names)>0):
        reg = reg[:-1]
    
    schedule_str = get_schedule_str_for_pruning(program_json, schedule_json)
    if re.search(reg, schedule_str):    
        return True
        
    return False
def get_schedule_str_for_pruning(program_json, sched_json):
    comp_name = [
        n
        for n in sched_json.keys()
        if not n in ["unfuse_iterators", "tree_structure", "execution_times", "fusions", "sched_str", "legality_check", "exploration_method"]
    ]
    sched_str = ""
#     print(f"starting for new schedules in program: {program_json}")
    for name in comp_name:
        # can probably use the feature in prog json
        transf_loop_nest = program_json["computations"][name]["iterators"].copy()
        schedule = sched_json[name]
        sched_str += '{' + name + '}:'
#         print(f"transf_loop_nest for computation: {name} is {transf_loop_nest}")
        if schedule["parallelized_dim"]:
            
            dim_index = transf_loop_nest.index(schedule["parallelized_dim"])
            sched_str += "P(L" + str(dim_index) + ")"

        if schedule["tiling"]:
            if schedule["tiling"]["tiling_depth"] == 2:
                first_dim = schedule["tiling"]["tiling_dims"][0]
                second_dim = schedule["tiling"]["tiling_dims"][1]
                
                first_dim_index = transf_loop_nest.index(first_dim)
                second_dim_index = transf_loop_nest.index(second_dim)
                first_factor = schedule["tiling"]["tiling_factors"][0]
                second_factor = schedule["tiling"]["tiling_factors"][1]
                sched_str += (
                    "T2(L"
                    + str(first_dim_index)
                    + ",L"
                    + str(second_dim_index)
                    + ","
                    + str(first_factor)
                    + ","
                    + str(second_factor)
                    + ")"
                )
                i = transf_loop_nest.index(first_dim)
                transf_loop_nest[i : i + 1] = first_dim + "_outer", second_dim + "_outer"
                i = transf_loop_nest.index(second_dim)
                transf_loop_nest[i : i + 1] = first_dim + "_inner", second_dim + "_inner"
            else:
                first_dim = schedule["tiling"]["tiling_dims"][0]
                second_dim = schedule["tiling"]["tiling_dims"][1]
                third_dim = schedule["tiling"]["tiling_dims"][2]
                first_dim_index = transf_loop_nest.index(first_dim)
                second_dim_index = transf_loop_nest.index(second_dim)
                third_dim_index = transf_loop_nest.index(third_dim)
                first_factor = schedule["tiling"]["tiling_factors"][0]
                second_factor = schedule["tiling"]["tiling_factors"][1]
                third_factor = schedule["tiling"]["tiling_factors"][2]
                sched_str += (
                    "T3(L"
                    + str(first_dim_index)
                    + ",L"
                    + str(second_dim_index)
                    + ",L"
                    + str(third_dim_index)
                    + ","
                    + str(first_factor)
                    + ","
                    + str(second_factor)
                    + ","
                    + str(third_factor)
                    + ")"
                )
                i = transf_loop_nest.index(first_dim)
                transf_loop_nest[i : i + 1] = (
                    first_dim + "_outer",
                    second_dim + "_outer",
                    third_dim + "_outer",
                )
                i = transf_loop_nest.index(second_dim)
                transf_loop_nest[i : i + 1] = (
                    first_dim + "_inner",
                    second_dim + "_inner",
                    third_dim + "_inner",
                )
                transf_loop_nest.remove(third_dim)

        if schedule["unrolling_factor"]:
            dim_index = len(transf_loop_nest) - 1
            dim_name = transf_loop_nest[-1]
            sched_str += "U(L" + str(dim_index) + "," + schedule["unrolling_factor"] + ")"
            transf_loop_nest[dim_index : dim_index + 1] = (
                dim_name + "_Uouter",
                dim_name + "_Uinner",
            )
    return sched_str
    
def add_expression_representation_to_function(program_json, function_name):
    # Checking if it already contains the expression representation
    if 'expression_representation' in program_json["computations"][list(program_json["computations"].keys())[0]]:
        return program_json
    
    # Looking for the function expressions file
    i=0
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/5x_functions/"+function_name+"_expr_representation.json")
    except:
        i+=1
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/depricated/"+function_name+"_expr_representation.json")
    except:
        i+=1
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/done/"+function_name+"_expr_representation.json")
    except:
        i+=1
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/done_mats/"+function_name+"_expr_representation.json")
    except:
        i+=1
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/done_mats_2comps/"+function_name+"_expr_representation.json")
    except:
        i+=1
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/done_mats_gnrl_mcomp/"+function_name+"_expr_representation.json")
    except:
        i+=1
    try:
        exprs_file = pd.read_json("/data/mk8958/exprs_jsons/filtered_function602486/"+function_name+"_expr_representation.json")
    except:
        i+=1
    if i == 7 :
        raise ComputationExpressionException
    
    # Adding the expression representation to each computation
    for comp in program_json["computations"]:
        if exprs_file[function_name+"_explored_schedules.json"]["computations"][comp] is not None:
            program_json["computations"][comp]["expression_representation"] = exprs_file[function_name+"_explored_schedules.json"]["computations"][comp]
        else:
            raise ComputationExpressionException
    return program_json
